### Code réalisé par l'équipe 8.
### Équipe : Anas Bouhemhem 1953369
###          Bouh Abdillahi 1940646

Laboratoire 2


In [1]:
import numpy as np
import pandas as pd 
import yaml
from yaml.loader import SafeLoader
import random
import math
import matplotlib.pyplot as plt
import operator

import os
from pathlib import Path

# Inclusion du fichier de pathloss du 3GPP (Méthodologie 1)
from f3GPP import *



e:\Documents\Hiver_2023\ELE8702\Lab2\f3GPP.py:60: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
  PLPrime_UMaNLOS = 13.54 + 39.08 * math.log10(d_3D) + 20 * math.log10(f_c) - 0.6 (h_UT - 1.5)
e:\Documents\Hiver_2023\ELE8702\Lab2\f3GPP.py:91: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
  PLPrime_UMiNLOS = 22.4 + 35.4 * math.log10(d_3D) + 21.3 * math.log10(f_c) - 0.3 (h_UT - 1.5)


## Section Lecture des fichiers d'entrés

#### Lecture du fichier cas8.yaml

In [2]:
# import pyyaml module
# Open the file and load the file
with open('cas8.yaml') as f:
    dataCas8 = yaml.load(f, Loader=SafeLoader)
    print(dataCas8)




{'ETUDE_IMPORTANT': {'SCENARIO': 'Uma', 'ANT_COORD_GEN': 'g', 'UE_COORD_GEN': 'a', 'COORD_FILES': {'read': 'data/coordonnees_case9.txt'}, 'DEVICES': {'Antenna1': {'number': 25}, 'UE1-App1': {'number': 30}, 'UE2-App2': {'number': 20}}, 'GEOMETRY': {'Surface': {'rectangle': {'length': 5000, 'height': 4000}}}}}


#### Lecture du fichier devices_db.yaml

In [3]:
# import pyyaml module
# Open the file and load the file
with open('devices_db.yaml') as f:
    dataDevices_db = yaml.load(f, Loader=SafeLoader)
    print(dataDevices_db)

{'ANTENNAS': {'Antenna1': {'type': 'antenna', 'name': 'Antenna1', 'height': 10, 'scenario': 'UMi', 'frequency': 28, 'gain': 40}, 'Antenna2': {'type': 'antenna', 'name': 'Antenna2', 'height': 35, 'scenario': 'RMa', 'frequency': 0.9, 'gain': 45}, 'Antenna3': {'type': 'antenna', 'name': 'Antenna1', 'height': 10, 'scenario': 'UMi', 'frequency': 28, 'gain': 45}, 'Antenna4': {'type': 'antenna', 'name': 'Antenna2', 'height': 37, 'scenario': 'RMa', 'frequency': 0.9, 'gain': 45}}, 'UES': {'UE1-App1': {'type': 'ue', 'name': 'UE1-App1', 'app': 'app1', 'height': 1.5}, 'UE2-App2': {'type': 'ue', 'name': 'UE2-App2', 'app': 'app2', 'height': 1.5}}}


## Création de fichier de coordonnées

### Routine de génération de coordonnées de la structure en grille 

In [4]:
def fill_up_the_lattice(N, lh, lv, nh, nv):
    """Function appelée par get_rectangle_lattice_coords()"""
    
    def get_delta1d(L, n):
        return L/(n + 1)
    
    coords = []
    deltav = get_delta1d(lv, nv)
    deltah = get_delta1d(lh, nh)
    line = 1
    y = deltav
    count = 0
    while count < N:
        if count + nh < N:
            x = deltah
            for  i in range(nh):
                # Fill up the horizontal line
                coords.append((x,y))
                x = x + deltah
                count += 1
            line += 1
        else:
            deltah = get_delta1d(lh, N - count)
            x = deltah
            for i in range(N - count):
                # Fill up the last horizontal line
                coords.append((x,y))
                x = x + deltah
                count += 1
            line += 1
        y = y +deltav
    return coords



def get_rectangle_lattice_coords(lh, lv, N, Np, nh, nv):
    """Function appelee par gen_lattice_coords()"""
    
    if Np > N:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv)
    elif Np < N:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv + 1)
    else:
        coords = fill_up_the_lattice(N, lh, lv, nh, nv)
    return coords


# In[170]:


def gen_lattice_coords(terrain_shape: dict, N: int):
    """Génère un ensemble de N coordonnées placées en grille 
       sur un terrain rectangulaire
    
       Args: terrain_shape: dictionary {'rectangle': {'length' : lh,
                                                   'height' : lv}
           lh and lv are given in the case file"""

    shape = list(terrain_shape.keys())[0]
    lh = terrain_shape[shape]['length']
    lv = terrain_shape[shape]['height']
    R = lv / lh    
    nv = round(math.sqrt(N / R))
    nh = round(R * nv)
    Np = nh * nv
    if shape.lower() == 'rectangle':
        coords = get_rectangle_lattice_coords(lh, lv, N, Np, nh, nv)
    else:
        msg = [f"\tImproper shape ({shape}) used in the\n",
                "\tgeneration of lattice coordinates.\n"
                "\tValid values: ['rectangle']"]
        ERROR(''.join(msg), 2)
    return coords

### Routine des coordonnées en grille

In [5]:
shape= {'rectangle': {'length': 200, 'height': 200}}
N = 32
coords = gen_lattice_coords(shape, N)
print(coords)

[(28.571428571428573, 28.571428571428573), (57.142857142857146, 28.571428571428573), (85.71428571428572, 28.571428571428573), (114.28571428571429, 28.571428571428573), (142.85714285714286, 28.571428571428573), (171.42857142857144, 28.571428571428573), (28.571428571428573, 57.142857142857146), (57.142857142857146, 57.142857142857146), (85.71428571428572, 57.142857142857146), (114.28571428571429, 57.142857142857146), (142.85714285714286, 57.142857142857146), (171.42857142857144, 57.142857142857146), (28.571428571428573, 85.71428571428572), (57.142857142857146, 85.71428571428572), (85.71428571428572, 85.71428571428572), (114.28571428571429, 85.71428571428572), (142.85714285714286, 85.71428571428572), (171.42857142857144, 85.71428571428572), (28.571428571428573, 114.28571428571429), (57.142857142857146, 114.28571428571429), (85.71428571428572, 114.28571428571429), (114.28571428571429, 114.28571428571429), (142.85714285714286, 114.28571428571429), (171.42857142857144, 114.28571428571429), (

### Calcul de Pathloss

In [6]:
def calcul_pathloss(distance_ue_antenne):
    """Calculate the pathloss (substitution for the 3gpp formulas)"""
    if  (distance_ue_antenne == 0) :
        pathloss=500*random.random()
    else : 
        pathloss = distance_ue_antenne
    return pathloss


### Section Lab 2

In [7]:
#pathCoord = os.path.abspath("") + '\\' + "test" 

# Lecture de fichier coordonées si existant:

isExistingCoord = os.path.isfile("coords_lab2_eq8.txt")
print(isExistingCoord)
#if (isExistingCoord == false):
    #création du fichier.


# Define Data : 

print(dataCas8['ETUDE_IMPORTANT']['COORD_FILES'])

if 'write' in dataCas8['ETUDE_IMPORTANT']['COORD_FILES']:
    print("Succes!")
else :
    print("Failure!")
    


False
{'read': 'data/coordonnees_case9.txt'}
Failure!


## Section Antennes/UE


### Objet Antennes et UE (User Equipement)

In [8]:

class Antenna:
    
    def __init__(self, id):
        self.id = id #id de l'antenne (int)
        self.group = None # group défini dans la base de données (str)
        self.coords = None # tuple contenant les coordonnées (x,y) 
        self.assoc_ues = [] # liste avec les id des UEs associés à l'antenne
        self.scenario = None # scénario tel que lu du fichier de cas (str)
        self.gen = None # type de géneration de coordonnées: 'g', 'a', etc. (str)
        
    
class UE:
    
    def __init__(self, id, app_name):
        self.id= id #id de l'UE (int)
        self.group = None # group défini dans la base de données (str)
        self.coords=None # tuple contenant les coordonnées (x,y) 
        self.app=app_name #nom de l'application qui tourne dans le UE (str)
        self.assoc_ant=None #id de l'antenne associée à l'UE (int)
        self.los = True # LoS ou non (bool)
        self.gen = None # type de géneration de coordonnées: 'g', 'a', etc. (str)
       

### Creation d'objets Antennes/UE et determination des coords

In [46]:
def place_ue_and_antennas():

###################################################
#              Section Antenne   
##################################################

    shape = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']
    global Nb_antenne
    Nb_antenne = dataCas8['ETUDE_IMPORTANT']['DEVICES']['Antenna1']['number'] 
    
    global listAntenne # facon d'acceder globalement a la liste
    listAntenne = []

    for x in range(N):
        listAntenne.append(Antenna(x))
        


    # Generation des coordonnees
    coords_antennes = gen_lattice_coords(shape, N)
  
    for x in range(N):
        listAntenne[x].coords = ([item[0] for item in coords_antennes][x],
                                [item[1] for item in coords_antennes][x])



    #############################
    # Test a des fin pratiques, a effacer apres
    #############################
    
    coords_antennes_x = [item[0] for item in coords_antennes]
    coords_antennes_y = [item[1] for item in coords_antennes]

    


###################################################
#              Section UE   
##################################################

    global nb_ue
    nb_ue =  \
    dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number'] + \
    dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE2-App2']['number']


    nb_ue_app_1 = dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number']
    nb_ue_app_2 = dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE2-App2']['number']
    
    coords_ue_x = []
    coords_ue_y = []
    coords_ue = []

    #*******************************************
    #   Creation des objets UE
    #*******************************************

    global listUE # facon d'acceder globalement a la liste (voir Antenne)
    listUE = []

    #Qte de UE app 1

    for x in range(nb_ue_app_1):
        listUE.append(UE(x, 'UE1-App1'))

    #Qte de UE app 2

    for x in range(nb_ue_app_2):
        listUE.append(UE(nb_ue_app_1 + x, 'UE2-App2'))


    #grid element layout generation and attribution

    for x in range(nb_ue):
        temp = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']['rectangle']['length'] \
            *random.random()
        coords_ue_x.append(temp)
        temp2 = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']['rectangle']['height'] \
            *random.random()
        coords_ue_y.append(temp2)

    #index = 0
    #while index < nb_ue:
    #    coords_ue.append((coords_ue_x[index], coords_ue_y[index]))
    #    index = index + 1


    # Attribution des coords dans les UE

    for x in range(len(listUE)):
        setattr(listUE[x], "coords", (coords_ue_x[x], coords_ue_y[x]))
    print(coords_ue)


    #calcul de distance 
    distance = []

    return listAntenne, listUE




Section de test d'affichage


In [10]:
shape = dataCas8['ETUDE_IMPORTANT']['GEOMETRY']['Surface']
#N = dataCas8['ETUDE_IMPORTANT']['DEVICES']['Antenna1']['number'] + \
#dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE1-App1']['number'] + \
#dataCas8['ETUDE_IMPORTANT']['DEVICES']['UE2-App2']['number']

#place_ue_and_antennas()
#print(shape)

#test = Antenna(1)
#print(test.coords)

#test2 = UE(3,"test")
#print("test 2 id = ", test2.id)
#print("test 2 group = ", test2.group)
#print("test 2 app name = ", test2.app)

place_ue_and_antennas()

#setattr(listAntenne[6], "coords", (1,2))
print(listUE[6].__dict__)

for x in range(len(listUE)):
    print(listUE[x].coords)    




[]
{'id': 6, 'group': None, 'coords': (4336.2910653665695, 3757.8450643718943), 'app': 'UE1-App1', 'assoc_ant': None, 'los': True, 'gen': None}
(2741.4319017324956, 1712.0404796286014)
(683.0039714301716, 2878.968710892877)
(1691.7936043527116, 3438.456871699966)
(717.9497702523863, 2275.953911698688)
(3842.4123927826886, 2430.7311260148117)
(2699.6257852388117, 3101.118404191008)
(4336.2910653665695, 3757.8450643718943)
(3545.091303926864, 3003.5799108473007)
(810.8383146612657, 2508.0646384914035)
(4727.812479467804, 5.976838700125775)
(3353.3621388181227, 2981.8802198375524)
(468.7198163680101, 2164.1447290003316)
(2427.001997108892, 1591.667904775962)
(1954.2842690969924, 986.6543696401822)
(2720.140353557892, 2555.1601474297363)
(3841.9110225645936, 1553.9356375076636)
(1868.0913745104976, 1035.2848635067273)
(904.2589422537272, 169.0259596239585)
(4909.778913883755, 1431.1469552774354)
(4716.908557716682, 2684.2142761316636)
(1008.361690498099, 684.7016819059891)
(4758.8479803095

In [64]:
#def calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne):

def calcul_association_ue_antenne(pathloss_computation):
     
    index_antennes = 0
    index_ue = 0
    smallestDistance = [0] * nb_ue
    check = 0
    calculPathlossChaqueAntenne = [] * Nb_antenne # nom a changer
    pathlossTotal = [] * nb_ue 


    for x in range(nb_ue):
        for y in range(Nb_antenne):
            temp = tuple(map(operator.sub, listUE[y].coords, listAntenne[y].coords))


            if pathloss_computation == "random" :
                check = calcul_pathloss(0) #random pathloss
                calculPathlossChaqueAntenne.append(calcul_pathloss(0))
                

            if pathloss_computation == "distance" :
                check = calcul_pathloss(math.sqrt(math.pow(temp[0],2) + math.pow(temp[1],2))) #pathloss from the distance
                calculPathlossChaqueAntenne.append(check)
                
            if y == 0: # Assigner premiere valeur
                path_loss_min = check 
                smallestDistance[x] = (check, y) #antenne commence a 1, et non a 0.
            elif check < path_loss_min: #sinon comparer pour la plus petite
                path_loss_min = check
                smallestDistance[x] = (check, y) #antenne commence a 1, et non a 0.
                temporary_value = y

        #print(calculPathlossChaqueAntenne)
        pathlossTotal.append(calculPathlossChaqueAntenne)
        calculPathlossChaqueAntenne = [] * Nb_antenne # nom a changer
        setattr(listUE[x], "assoc_ant", (temporary_value))

    
    # print(pathlossTotal)

    return smallestDistance, pathlossTotal               
# commented region
    #     while index_ue < nb_ue:
    #         index_antennes = 0
    #         path_loss_min = 10000000000 #valeur arbitraire, il faut juste qu'elle soit assez grande pour prendre la valeur la plus petite.
    #         while index_antennes < N:
    #             temp = tuple(map(operator.sub, coords_ue[index_ue], coords_antennes[index_antennes]))
                
    #             if pathloss_computation == "random" :
    #                 check = calcul_pathloss(0) #random pathloss
    #             if pathloss_computation == "distance" :
    #                 check = calcul_pathloss(math.sqrt(math.pow(temp[0],2) + math.pow(temp[1],2))) #pathloss from the distance
    #             #check if pathloss is the smallest one in the network of antennas
    #             if check < path_loss_min:
    #                 path_loss_min = check
    #                 smallestDistance[index_ue] = (check,index_antennes + 1) #antenne commence a 1, et non a 0.

    #             index_antennes = index_antennes + 1
                
    #         index_ue = index_ue + 1



    #     numerant = []
    #     pathloss = []
    #     id_ue = []
    #     temp = 1
    #     for i in smallestDistance:
    #         numerant.append(i[1])
    #         pathloss.append(i[0])
    #         id_ue.append(temp)
    #         temp = temp + 1
    #     return id_ue, numerant, pathloss



    # def inv_calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne):
    #     nb_ue = N_ue 
    #     index_antennes = 0
    #     index_ue = 0
    #     N = N_antenne
    #     smallestDistance = [0] * N
    #     while index_antennes < N: 
    #         index_ue = 0
    #         path_loss_min = 10000000000 #valeur arbitraire, il faut juste qu'elle soit assez grande pour prendre la valeur la plus petite.
    #         while index_ue < nb_ue:
    #             temp = tuple(map(operator.sub, coords_ue[index_ue], coords_antennes[index_antennes]))
                
    #             if pathloss_computation == "random" :
    #                 check = calcul_pathloss(0) #random pathloss
    #             if pathloss_computation == "distance" :
    #                 check = calcul_pathloss(math.sqrt(math.pow(temp[0],2) + math.pow(temp[1],2))) #pathloss from the distance
    #             #check if pathloss is the smallest one in the network of antennas
    #             if check < path_loss_min:
    #                 path_loss_min = check
    #                 smallestDistance[index_antennes] = (check,index_ue + 1) #ue commence a 1, et non a 0.

    #             index_ue = index_ue + 1
                
    #         index_antennes = index_antennes + 1


    #     numerue = []
    #     pathloss = []
    #     id_ant = []
    #     temp = 1
    #     for i in smallestDistance:
    #         numerue.append(i[1])
    #         pathloss.append(i[0])
    #         id_ant.append(temp)
    #         temp = temp + 1

    #     return id_ant, numerue, pathloss
# endregion

### Section Test 

In [67]:
smallestDistance, pathlossTotal = calcul_association_ue_antenne("random")
print(smallestDistance)
print(listUE[5].__dict__)

[(6.92294886915823, 11), (5.995637550726851, 4), (25.99597249056157, 7), (14.358899026644556, 24), (13.221095460800413, 0), (14.363585349699948, 7), (14.715401236586601, 19), (11.785877362634755, 0), (12.030499921896087, 23), (20.35878349322806, 5), (4.877733604222501, 2), (18.42377172791504, 14), (13.145878025000501, 4), (15.048106225746983, 19), (28.316675248984957, 4), (40.634391204052044, 4), (0.8622606847593639, 17), (5.0129938926820605, 3), (3.91827308178333, 24), (54.055763179302, 17), (27.15229129776614, 19), (11.441452333805668, 1), (66.9976163104648, 6), (52.6460084393266, 23), (15.675796123528707, 1), (84.31220955865543, 1), (5.484200285844287, 15), (13.354006224617132, 8), (22.950882460268552, 17), (8.244531409881406, 8), (5.886488681993707, 17), (31.075233854488793, 19), (23.349593424575655, 16), (4.22262005104912, 17), (37.42981416832908, 10), (54.59840526577192, 12), (19.68234988227191, 15), (0.16415721614410073, 20), (52.55676995611752, 11), (39.39554093640424, 9), (2.0

### Écriture dans un fichier

In [ ]:
def write_file_assoc(pathloss_computation, pl_lab1_rand_eqn, assoc_lab1_rand_eqn, coords_ue, coords_antennes, N_ue, N_antenne):

    id_ue, numerant, pathloss = calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne)


    data = {"ID UE": id_ue, "ID antenne": numerant,
    "Pathloss entre UE et antenne": pathloss
    }

    id_ant, numerue, pathloss = inv_calcul_association_ue_antenne(pathloss_computation, coords_ue, coords_antennes, N_ue, N_antenne)
    data_inv = {"ID antenne": id_ant, "ID UE": numerue, 
    "Pathloss entre UE et antenne": pathloss
    }

    df = pd.DataFrame(data=data)
    df_inv = pd.DataFrame(data=data_inv)

    grp = df.sort_values(by=['ID antenne'])
    new_df = grp['ID antenne'].tolist()
    new_df_id_ue = grp['ID UE'].tolist()
    list_antenne_sorted = list(range(N_antenne + 1))
    list_antenne_sorted = list_antenne_sorted[1:]
    buffer = []
    vector = [0]*N_antenne
    current_ue_selection = ""

    #This let's us associate an antenna to a list of specified UEs
    temp = 0
    for j in  id_ue :
        if temp == 0 and j == 0 :
            current_ue_selection = current_ue_selection + str(new_df_id_ue[j-1] ) + " "
        if j < 300 and j > 0 :
            if new_df[j-1] == new_df[j] and j < 300 and j > 0 :
                current_ue_selection = current_ue_selection + str(new_df_id_ue[j-1]) + " "
            else :
                buffer.append(current_ue_selection)
                vector[temp] = buffer 
                buffer = []
                current_ue_selection = ""
                temp = temp + 1

        if temp == (N_antenne-1) :
            if (j == N_ue):
                current_ue_selection = current_ue_selection + " " + str(new_df_id_ue[j-1])
                buffer.append(current_ue_selection)
                vector[temp] = buffer 

    #Dataframe to encapsulate the association antenna to list of UEs
    data_assoc = { "ID antenne": list_antenne_sorted, "List ID UE": vector
    }
    df_assoc = pd.DataFrame(data=data_assoc)
    
    #Writing necessary files
    path = os.path.abspath("") + '\\' + pl_lab1_rand_eqn 
    path_inv = os.path.abspath("") + '\\' + assoc_lab1_rand_eqn 
    with open(path, 'w') as f:
        df_string = df.to_string(header=False, index=False)
        f.write(df_string)
    f.close()
    with open(path_inv, 'w') as f_inv:
        df_string_inv = df_assoc.to_string(header=False, index=False)
        f_inv.write(df_string_inv)
    f_inv.close()



### Fonction lab1_eq8


In [55]:
def lab1_eq8() :
    # N_ue = 300
    # N_antenne = 25
    listAntenne, listUE = place_ue_and_antennas()

    user_choose_name = input("Voulez-vous choisir le nom des fichiers à ecrire? ")   

    #pathloss random
    if user_choose_name == "oui" :
        pl_lab1_eqn = input("Quel nom de fichier pour pl_lab1_rand_eqn.txt ")
        assoc_lab1_eqn = input("Quel nom de fichier pour assoc_lab1_rand_eqn.txt ")
    else :
        pl_lab1_eqn = "pl_lab1_rand_eq8.txt"
        assoc_lab1_eqn = "assoc_lab1_rand_eq8.txt" 

    pathloss_computation = "random"
    # write_file_assoc(pathloss_computation, pl_lab1_eqn, assoc_lab1_eqn, coords_ue, coords_antennes, N_ue, N_antenne)

    #pathloss proportional to distance
    if user_choose_name == "oui" :
        pl_lab1_eqn = input("Quel nom de fichier pour pl_lab1_dist_eqn.txt ")
        assoc_lab1_eqn = input("Quel nom de fichier pour assoc_lab1_dist_eqn.txt ")
    else :
        pl_lab1_eqn = "pl_lab1_dist_eq8.txt"
        assoc_lab1_eqn ="assoc_lab1_dist_eq8.txt"

    pathloss_computation = "distance"
    # write_file_assoc(pathloss_computation, pl_lab1_eqn, assoc_lab1_eqn, coords_ue, coords_antennes, N_ue, N_antenne)


    return listAntenne, listUE


### Fonction lab1()

In [60]:
def lab1() :
    listAntenne, listUE = lab1_eq8()
    print(listUE[5].__dict__)
    # return listAntenne, listUE
    

### Fonction Main

In [63]:
lab1()

[]
{'id': 5, 'group': None, 'coords': (438.3777397823857, 1221.2421635857088), 'app': 'UE1-App1', 'assoc_ant': None, 'los': True, 'gen': None}


## Traitement de scénario de propagation 3GPP (UMi, UMa, etc.)

Le module python qui fait les calculs de pathloss du 3GPP. Veillez l'appeler ```calcul_pathloss_3gpp_eqn.py```

## Calcul de pathloss basés sur ces scénarios
Faire des calculs de pathloss basés sur ces scénarios


## Association des UEs pour chaque antenne
Pour chaque UE, montrer l'antenne associée

## Montrer la liste des UEs associés pour CHAQUE Antenne
Pour chaque antenne, montrer la liste des UEs qu'y sont associés.

## Fournir les fichiers suivants :


    Le fichier de cas utilisé, appelé ```cas_lab2_eqn.yaml```
    Le fichier de coordonnées, appelé ```coords_lab2_eqn.txt``` (voir format dans section méthodologie)
    Le fichier de visibilité, appelé ```visibility_lab2_eqn.txt``` (voir format dans exemple ci-dessus)
    Un fichier d'association, appelé ```assoc_ant_lab2_eqn.txt``` qui montre pour chaque antenne, la liste des UEs associées (voir format dans livrables Lab1)
    Un fichier d'association, appelé ```assoc_ue_lab2_eqn.txt``` qui montre pour chaque UE, l'antenne associé
    Le fichier de pathloss, appelé ```pl_lab2_eqn.txt```  qui montre le pathloss pour chaque UE et antenne  (voir format dans livrables Lab1)